In [ ]:
import ethereumetl
import pandas as pd
import numpy as np
import os

In [ ]:
# Set Sepolia RPC
provider_uri = 'YOUR_SEPOLIA_RPC_ENDPOINT'

In [ ]:
# Last block of each calendar year
last_block_2021 = 345734
last_block_2022 = 2297135 # Through Nov 15, 2022

In [ ]:
# 2021 contract deployers
start_block = 1
end_block = start_block + 99999

contract_deployers = []

while (end_block <= last_block_2021+100000): # Buffer based on chunk size
    end_block_used = min(end_block,last_block_2021)
    print("Fetching blocks "+str(start_block)+ " to "+str(end_block_used)+".")
    ! ethereumetl export_blocks_and_transactions --start-block $start_block --end-block $end_block_used \
    --provider-uri $provider_uri \
    --transactions-output transactions.csv

    transactions = pd.read_csv('transactions.csv')
      
    # Get contract deployers
    transactions = transactions[transactions['to_address'].isnull()]
    contract_deployers = contract_deployers + transactions['from_address'].unique().tolist()
    
    # Delete tempfile and increment block range
    os.remove("transactions.csv")
    start_block = start_block + 100000
    end_block = end_block + 100000

# Remove duplicates across chunks
unique_contract_deployers = [*set(contract_deployers)]

# Export to CSV
unique_contract_deployers_output = pd.DataFrame(unique_contract_deployers, columns=["contract_deployers"])
unique_contract_deployers_output.to_csv('sepolia_unique_contract_deployers_2021.csv', index=False)

print("Unique Contract Deployers (2021): "+str(len(unique_contract_deployers)))

In [ ]:
#2022 contract deployers
start_block = last_block_2021+1
end_block = start_block + 99999

contract_deployers = []

while (end_block <= last_block_2022+100000): # Buffer based on chunk size
    end_block_used = min(end_block,last_block_2022) # Through Nov 15, 2022
    print("Fetching blocks "+str(start_block)+ " to "+str(end_block_used)+".")
    ! ethereumetl export_blocks_and_transactions --start-block $start_block --end-block $end_block_used \
    --provider-uri $provider_uri \
    --transactions-output transactions.csv

    transactions = pd.read_csv('transactions.csv')
      
    # Get contract deployers
    transactions = transactions[transactions['to_address'].isnull()]
    contract_deployers = contract_deployers + transactions['from_address'].unique().tolist()
    
    # Delete tempfile and increment block range
    os.remove("transactions.csv")
    start_block = start_block + 100000
    end_block = end_block + 100000

# Remove duplicates across chunks
unique_contract_deployers = [*set(contract_deployers)]

# Export to CSV
unique_contract_deployers_output = pd.DataFrame(unique_contract_deployers, columns=["contract_deployers"])
unique_contract_deployers_output.to_csv('sepolia_unique_contract_deployers_2022.csv', index=False)

print("Unique Contract Deployers (2022): "+str(len(unique_contract_deployers)))

In [ ]:
##### ANALYSIS

In [ ]:
contract_deployers_2021 = pd.read_csv('sepolia_unique_contract_deployers_2021.csv')
contract_deployers_2022 = pd.read_csv('sepolia_unique_contract_deployers_2022.csv')

# Generate CSV for full time range
contract_deployers_all = contract_deployers_2021.append(contract_deployers_2022)
unique_contract_deployers_all = contract_deployers_all['contract_deployers'].unique()
unique_contract_deployers_all_df = pd.DataFrame(unique_contract_deployers_all, columns=['contract_deployers'])
unique_contract_deployers_all_df.to_csv('sepolia_unique_contract_deployers_all.csv', index=False)

# Calculate Scores: 1 point for each calendar year in which an address deployed a contract
sepolia_scores = contract_deployers_all.assign(Count=1).groupby('contract_deployers')['Count'].count().reset_index()

# Show score distribution
sepolia_scores_summary = sepolia_scores.groupby('Count').count()
display(sepolia_scores_summary)

In [ ]:
# Chart of unique contract deployers over time
year = ["2021","2022"]
deployers = [len(contract_deployers_2021.index),len(contract_deployers_2022.index)]

df = pd.DataFrame({'contract deployers': deployers}, index=year)

axes = df.plot.bar(rot=0, subplots=True)